# Initial State Configuration for FCFS Queue with Exponential Service

This example demonstrates initial state configuration for transient analysis of a closed queueing network with FCFS scheduling and exponential service times.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.ctmc.*
import jline.solvers.ssa.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)

// Create closed class with 5 jobs
val class1 = ClosedClass(model, "Class1", 5, queue1, 0)  // Initial jobs start at Queue1

// Set service processes
delay.setService(class1, Exp(1.0))   // λ = 1.0 for delay station
queue1.setService(class1, Exp(0.7))  // μ = 0.7 for queue station

In [ ]:
// Set up circular routing between the two stations
val P = model.initRoutingMatrix()

// Circular routing: Delay → Queue1 → Delay
P.set(class1, class1, delay, queue1, 1.0)
P.set(class1, class1, queue1, delay, 1.0)

model.link(P)

val M = model.numberOfStations
val K = model.numberOfClasses

println("Model Configuration:")
println("- Number of stations: $M")
println("- Number of classes: $K")
println("- Total jobs: ${class1.population}")
println("- Initial placement: All jobs start at ${queue1.name}")

In [ ]:
// Solve with MVA for steady-state analysis
val solverMva = MVA(model)
val avgTableMva = solverMva.avgTable

println("\n=== MVA Steady-State Results ===")
avgTableMva.print()

// Calculate utilizations
println("\nSystem Analysis:")
val throughput = solverMva.avgThroughput.get(0, 0)  // Class 1 throughput
val delayUtil = throughput / 1.0  // ρ_delay = λ * E[S] = throughput * 1.0
val queueUtil = throughput / 0.7  // ρ_queue = λ * E[S] = throughput * (1/0.7)

println("- Throughput: $throughput")
println("- Delay utilization: $delayUtil")
println("- Queue utilization: $queueUtil")
println("- Bottleneck: ${if (queueUtil > delayUtil) "Queue1 (slower service)" else "Delay"}")

In [ ]:
// Note: Transient analysis with initial state configuration
// would require specialized solvers and initial state setup
println("\n=== Initial State Analysis Notes ===")
println("This model demonstrates the setup for transient analysis where:")
println("- All 5 jobs initially start at Queue1 (position 0)")
println("- In transient analysis, we would observe how the system")
println("  evolves from this initial state to steady state")
println("- The initial queue lengths would be [0, 5] for [Delay, Queue1]")
println("- Over time, jobs redistribute according to the service rates")
println("\nKey Transient Behaviors:")
println("- Initially: Queue1 is saturated, Delay is empty")
println("- Transient: Jobs gradually move to Delay station")
println("- Steady-state: Jobs distributed as shown in MVA results above")
println("\nFor full transient analysis, specialized solvers like:")
println("- CTMC (Continuous Time Markov Chain)")
println("- SSA (Stochastic Simulation)")
println("- FLD (Fluid approximation)")
println("would be used with proper initial state configuration.")

This example demonstrates:

1. **Initial State Configuration**: 
   - All 5 jobs start at Queue1 (non-equilibrium initial state)
   - System must evolve from this initial placement to steady state
2. **Closed Network Dynamics**:
   - Circular routing between Delay and Queue1
   - Fixed population of 5 jobs
   - Queue1 has slower service (μ=0.7) than Delay (μ=1.0)
3. **Transient vs Steady-State**:
   - Initial: Severe imbalance (all jobs at Queue1)
   - Transient: Gradual redistribution
   - Steady-state: Balanced distribution based on service rates
4. **Performance Implications**:
   - Queue1 is the system bottleneck (higher utilization)
   - Initial response times would be very high
   - System performance improves as it approaches steady state

This type of analysis is important for:
- System startup behavior
- Recovery from failures or maintenance
- Understanding transient performance degradation
- Optimizing initial job placement strategies